In [1]:
# %%
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt

# %%
import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3,4,5,6,7,8])

# %%
os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf

# %%
import glob

path = r'H:\all\RL_Shrinkage_2024\ONE_YR_long_only\NonLinear_Shrinkage\transformed_qis_eigenvalues'
extension = 'csv'
os.chdir(path)
filenames = glob.glob('*.{}'.format(extension))
print(filenames)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\traitlets\con

['cov2para_evs_df.csv', 'cov2para_fixed04_evs_df.csv', 'cov2para_plus02_evs_df.csv', 'qis_evs_df.csv', 'qis_evs_df_p100.csv', 'qis_evs_df_p225.csv', 'qis_evs_df_p30.csv', 'qis_evs_df_p50.csv', 'qis_evs_df_p500.csv', 'qis_evs_exp_05.csv', 'qis_evs_exp_2.csv', 'sample_evs_df.csv', 'sample_evs_df_p100.csv', 'sample_evs_df_p225.csv', 'sample_evs_df_p30.csv', 'sample_evs_df_p50.csv', 'sample_evs_df_p500.csv']


In [2]:
    def get_new_eigenvalues(qis_eigenvalues, sample_eigenvalues, intensity_of_intensity):
        intensity = qis_eigenvalues.copy()
        if qis_eigenvalues.shape[1] == 500:
            intensity.iloc[: , -251:] = qis_eigenvalues.iloc[: , -251:] / sample_eigenvalues.iloc[: , -251:]
        else:
            intensity = qis_eigenvalues / sample_eigenvalues
        intensity_delta = intensity - 1
        intensity_delta_new = intensity_delta * intensity_of_intensity
        intensity_new = intensity_delta_new + 1
        qis_evs_new = intensity_new * sample_eigenvalues
        if qis_eigenvalues.shape[1] == 500:
            qis_evs_new.iloc[: , 0:250] = intensity_of_intensity * qis_eigenvalues.iloc[: , 0:250]

        # get Rotation Points
        intens_df = qis_eigenvalues / sample_eigenvalues
        if qis_eigenvalues.shape[1] == 500:  # only 1 year case ofcourse
            intens_df = intens_df.fillna(0)
        right_rotation_idx = np.argmin( np.abs(intens_df-1) , axis=1)
        left_rotation_idx = right_rotation_idx - 1
        rotation_point_sample_evs = 0.5 * np.diag(sample_eigenvalues.iloc[:, left_rotation_idx]) + 0.5 * np.diag(sample_eigenvalues.iloc[:, right_rotation_idx])
        rotation_points = 0.5 * np.diag(intens_df.iloc[:, left_rotation_idx]) + 0.5 * np.diag(intens_df.iloc[:, right_rotation_idx])
        rotation_points = rotation_points * rotation_point_sample_evs

        # Force all eigenvalues to be in decreasing order (from the largest one)
        idx_is_increasing = qis_evs_new.diff(axis=1).fillna(0) < 0
        qis_evs_new[idx_is_increasing] = np.nan
        qis_evs_new = qis_evs_new.bfill(axis=1)

        # check if values left (right) of rotation point are smaller (larger) than rotation point
        # check: is idx < left and value_at_idx > value_at_left --> then np.nan and ffill it for left (and bfill for right)
        for i in range(qis_evs_new.shape[0]):
            tmp = qis_evs_new.iloc[i, :].copy()
            left_bool = tmp[0:left_rotation_idx[i]] > rotation_points[i]
            right_bool = tmp[left_rotation_idx[i]:] < rotation_points[i]
            # change those evs on right that are smaller than rotation point
            tmp_right = tmp[left_rotation_idx[i]:]
            tmp_right[right_bool] = np.nan
            tmp_right.bfill(inplace=True) 
            tmp_right.ffill(inplace=True)  # in case the largest ev is NAN, we need to additionally ffill
            # change those evs on left that are LARGER than rotation point
            tmp_left = tmp[0:left_rotation_idx[i]]
            tmp_left[left_bool] = np.nan
            tmp_left.ffill(inplace=True) 
            tmp_left.bfill(inplace=True) # in case the smallest ev is NAN, we need to additionally bfill
            tmp_left.fillna(rotation_points[i], inplace=True)  # in case all eigenvalues left of rotation point are larger than rotation point
            # change qis_evs_new row
            qis_evs_new.iloc[i, :] = np.concatenate([tmp_left, tmp_right])

        # a few correction checks
        assert any(qis_evs_new.diff(axis=1).fillna(0) < 0), "Eigenvalues are not monotonically decreasing!"
        assert any(qis_evs_new.isna()), "There are NaN's in the QIS Eigenvalue Matrix!"
    
        return qis_evs_new

In [3]:
def get_rawres_long_only(eigenvalue_dict, modelnames: list, idxlist: list):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    weights_dict = dict()
    for idx in idxlist:
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = hf.calc_global_min_variance_pf_long_only(sigmahat)
                weights_dict[idx] = weights
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

        if idx % 250 == 0:
            print(f"done {idx} out of {permnos.shape[0]}")

    return tmp_rawres, tmp_res, weights_dict

In [4]:
def get_rawres_long_short(eigenvalue_dict, modelnames: list, idxlist: list):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    weights_dict = dict()
    for idx in idxlist:
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = hf.calc_global_min_variance_pf(sigmahat)
                weights_dict[idx] = weights
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

        if idx % 250 == 0:
            print(f"done {idx} out of {permnos.shape[0]}")

    return tmp_rawres, tmp_res, weights_dict

In [5]:
PF_SIZE = 500
model_names = ["qis",  "sample"]
evs_dfs_LONG_ONLY = {}
qis_evs_path = r"H:\all\RL_Shrinkage_2024\ONE_YR_long_only\NonLinear_Shrinkage\transformed_qis_eigenvalues"
evs_dfs_LONG_ONLY["qis"] = pd.read_csv(qis_evs_path + f'\\qis_evs_df_p{PF_SIZE}.csv', index_col=0)
evs_dfs_LONG_ONLY["sample"] = pd.read_csv(qis_evs_path + f'\\sample_evs_df_p{PF_SIZE}.csv', index_col=0)

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR_long_only\preprocessing\rets_permnos_1Y\permnos_1Y_p{PF_SIZE}.pickle")

intensity_of_intensity_list = [0.5, 1.0, 1.5]
print("Calculating new Eigenvalues..")
qis_evs_new_LONG_ONLY = {}
intensities_new_LONG_ONLY = {}
for intensity_of_intensity in intensity_of_intensity_list:
    qis_evs_new_LONG_ONLY[intensity_of_intensity] = get_new_eigenvalues(evs_dfs_LONG_ONLY["qis"], evs_dfs_LONG_ONLY["sample"], intensity_of_intensity)
print("Finished Calculating new Eigenvalues")


Calculating new Eigenvalues..
Finished Calculating new Eigenvalues


In [6]:
PF_SIZE = 500
model_names = ["qis",  "sample"]
evs_dfs_LONG_SHORT = {}
qis_evs_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\transformed_qis_eigenvalues"
evs_dfs_LONG_SHORT["qis"] = pd.read_csv(qis_evs_path + f'\\qis_evs_df_p{PF_SIZE}.csv', index_col=0)
evs_dfs_LONG_SHORT["sample"] = pd.read_csv(qis_evs_path + f'\\sample_evs_df_p{PF_SIZE}.csv', index_col=0)

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{PF_SIZE}.pickle")

intensity_of_intensity_list = [0.5, 1.0, 1.5]
print("Calculating new Eigenvalues..")
qis_evs_new_LONG_SHORT = {}
intensities_new_LONG_SHORT = {}
for intensity_of_intensity in intensity_of_intensity_list:
    qis_evs_new_LONG_SHORT[intensity_of_intensity] = get_new_eigenvalues(evs_dfs_LONG_SHORT["qis"], evs_dfs_LONG_SHORT["sample"], intensity_of_intensity)
print("Finished Calculating new Eigenvalues")


Calculating new Eigenvalues..
Finished Calculating new Eigenvalues


In [7]:
### long only
if 1 == 1:
    import pickle
    base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
    # IMPORT SHRK DATASETS
    pf_size = PF_SIZE  # DONT CHANGE HERE!!
    permnos = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR_long_only\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
    rets_full = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR_long_only\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")

    fixed_shrk_name = 'cov1Para'
    opt_shrk_name = 'cov1Para'
    with open(rf"{base_folder_path}\ONE_YR_long_only\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
        fixed_shrk_data = pickle.load(f)
    with open(rf"{base_folder_path}\ONE_YR_long_only\preprocessing\training_dfs\PF{pf_size}\{opt_shrk_name}_factor-1.0_p{pf_size}.pickle", 'rb') as f:
        optimal_shrk_data = pickle.load(f)

    # get all the validation indices
    len_train = 5040
    end_date = fixed_shrk_data.shape[0]
    # temp here
    val_indices_correct = (len_train, end_date)
    val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
    val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
    reb_date_1 = permnos.index[0]
    add_idx = np.where(rets_full.index == reb_date_1)[0][0]

    # %%
    print(f"creating res and rawres matrices for PF SIZE: {PF_SIZE}..")
    idxlist = list(range(5040, 10353, 21))
    tmp_rawres, tmp_res, WEIGHTS_LONG_ONLY =  get_rawres_long_only(qis_evs_new_LONG_ONLY, modelnames = intensity_of_intensity_list, idxlist=idxlist)
    all_res_LONG_ONLY = pd.DataFrame(tmp_res.copy())
    all_rawres_LONG_ONLY = pd.DataFrame(tmp_rawres.copy())

creating res and rawres matrices for PF SIZE: 500..
done 5250 out of 10353


In [8]:
### long short
if 1 == 1:
    import pickle
    base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
    # IMPORT SHRK DATASETS
    pf_size = PF_SIZE  # DONT CHANGE HERE!!
    permnos = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
    rets_full = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")

    fixed_shrk_name = 'cov1Para'
    opt_shrk_name = 'cov1Para'
    with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
        fixed_shrk_data = pickle.load(f)
    with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\{opt_shrk_name}_factor-1.0_p{pf_size}.pickle", 'rb') as f:
        optimal_shrk_data = pickle.load(f)

    # get all the validation indices
    len_train = 5040
    end_date = fixed_shrk_data.shape[0]
    # temp here
    val_indices_correct = (len_train, end_date)
    val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
    val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
    reb_date_1 = permnos.index[0]
    add_idx = np.where(rets_full.index == reb_date_1)[0][0]

    # %%
    print(f"creating res and rawres matrices for PF SIZE: {PF_SIZE}..")
    idxlist = list(range(5040, 10353, 21))
    tmp_rawres, tmp_res, WEIGHTS_LONG_SHORT =  get_rawres_long_short(qis_evs_new_LONG_SHORT, modelnames = intensity_of_intensity_list, idxlist=idxlist)
    all_res_LONG_SHORT = pd.DataFrame(tmp_res.copy())
    all_rawres_LONG_SHORT = pd.DataFrame(tmp_rawres.copy())

creating res and rawres matrices for PF SIZE: 500..
done 5250 out of 10353


## Compare the weights
- also find out why the long short version takes so much longer..

In [14]:
all_rawres_LONG_SHORT.std() * np.sqrt(252) * 100

0.5    10.615772
1.0    10.376136
1.5    10.411982
dtype: float64

In [15]:
all_rawres_LONG_ONLY.std() * np.sqrt(252) * 100

0.5    12.148598
1.0    12.242208
1.5    12.359996
dtype: float64

In [17]:
WEIGHTS_LONG_SHORT[5040].sum()

np.float64(1.0)

In [31]:
WEIGHTS_LONG_SHORT[5040][ (WEIGHTS_LONG_SHORT[5040] < 1e-5) & (WEIGHTS_LONG_SHORT[5040] > -1e-5) ]

array([], dtype=float64)

In [18]:
WEIGHTS_LONG_ONLY[5040].sum()

np.float64(1.0000000000000007)

In [34]:
### transform weights long only

w_long_copies = {}
for k, v in WEIGHTS_LONG_ONLY.items():
    w_long_copies[k] = v.copy()
    w_long_copies[k][(w_long_copies[k] < 1e-4) & (w_long_copies[k] > -1e-4) ] = 0

In [37]:
w_long_copies[5040].sum()

np.float64(1.0000000000000007)

In [40]:
def GET_RAWRES_LONG_ONLY_TEMPORARY(eigenvalue_dict, modelnames: list, idxlist: list, input_weights: dict):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    weights_dict = dict()
    for idx in idxlist:

        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
        for cur_modelname in modelnames:
            try:
                weights = input_weights[idx]
                weights_dict[idx] = weights
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

        if idx % 250 == 0:
            print(f"done {idx} out of {permnos.shape[0]}")

    return tmp_rawres, tmp_res

In [43]:
tmp_rawres, tmp_res = GET_RAWRES_LONG_ONLY_TEMPORARY(qis_evs_new_LONG_ONLY, modelnames=[1.5], idxlist=idxlist, input_weights=WEIGHTS_LONG_ONLY)

done 5250 out of 10353


In [49]:
np.array(tmp_rawres[1.5]).std()*np.sqrt(252) * 100

np.float64(12.358832656698091)

In [44]:
### COMPARE QIS EVS LONG ONLY WITH QIS EVS USUAL CASE

In [51]:
import cvxpy as cp
def calc_global_min_variance_pf_long_only_TESTING(covmat_estimator):
    """
    Calculates the global minimum portfolio WITH SHORT SELLING CONSTRAINTS
    :param covmat_estimator: covariance matrix estimator of shape p x p
    :return: portfolio weights
    """
    # since I know that my covmat estimator is psd I can add the line below
    # usual optimization

    if isinstance(covmat_estimator, pd.DataFrame):
        covmat_estimator = covmat_estimator.values  # or .to_numpy()

    w = cp.Variable(covmat_estimator.shape[0])  
    risk = cp.quad_form(w, covmat_estimator, assume_PSD=True)
    objective = cp.Minimize(risk)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    # result is stored in the cp.Variable (w) after solving
    return w.value

In [52]:
def get_rawres_long_only_TEMPORARY_V2(eigenvalue_dict, modelnames: list, idxlist: list):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    weights_dict = dict()
    for idx in idxlist:
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = calc_global_min_variance_pf_long_only_TESTING(sigmahat)
                weights_dict[idx] = weights
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

        if idx % 250 == 0:
            print(f"done {idx} out of {permnos.shape[0]}")

    return tmp_rawres, tmp_res, weights_dict

In [59]:
tmp_rawres, tmp_res, weights_dict = get_rawres_long_only_TEMPORARY_V2(eigenvalue_dict=qis_evs_new_LONG_ONLY, modelnames=[0.5, 1.0, 1.5], idxlist=idxlist)

done 5250 out of 10353


In [61]:
pd.DataFrame(tmp_rawres).std()* np.sqrt(252)*100

0.5    12.148628
1.0    12.242847
1.5    12.359996
dtype: float64